    # This code snippet performs the following tasks:

    # Installs the Azure Machine Learning SDK package.
    # Imports necessary modules and packages required for working with Azure Machine Learning, handling files, and making HTTP requests.
    # Reads configuration data from a JSON file (config.json) containing subscription details, resource group information, workspace name, and region.
    # Creates an Azure Machine Learning workspace using the provided subscription details, resource group, workspace name, and region.
    # Registers a machine learning model (Diabetes_Pipeline.pkl) in the Azure Machine Learning workspace.
    # Defines a Conda environment (my-conda-env) with scikit-learn as a dependency.
    # Creates an inference configuration for the model, specifying the entry script (score.py) and the environment.
    # Specifies deployment configuration for Azure Container Instances (ACI), including CPU cores and memory.
    # Deploys the model as a web service using Azure Container Instances.
    # Retrieves the scoring URI of the deployed service.

In [ ]:
!pip install azureml-sdk

### **Importing the dependencies**

In [ ]:
import os
import json
import requests


from azureml.core import Workspace
from azureml.core import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice



In [ ]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"


# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]
workspace_name = data["workspace_name"]
region = data["region"]

In [ ]:
print(subscription_id)
print(resource_group)
print(workspace_name)
print(region)

**Create a Resource Group from Azure Portal**

In [ ]:
# Create a workspace

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

In [ ]:
# Specify the path to your  model file
model_path = 'Diabetes_Pipeline.pkl'

In [ ]:
model_name='Diabetes_Prediction_Model'

In [ ]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

In [ ]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn']) # setting up the environment for our model which is scikit learning here


In [ ]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [ ]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [ ]:
service = Model.deploy(workspace=ws,
                       name='diabetes-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

In [ ]:
scoring_uri = service.scoring_uri

In [ ]:
scoring_uri

**Cleaning up all the created resources**

In [ ]:
# Delete the service

service_name = "diabetes-prediction-service"

service = ws.webservices[service_name]
service.delete()

In [ ]:
# Specify the name of the registered model to delete
model_name = 'diabetes_prediction_model'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()

In [ ]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)

1. Azure Account with appropriate subscription
2. Configuration file that has subscription_id and other details like resource_group name, etc
3. Create a resource group from Azure Portal
4. Create a workspace
5. Register the model in container registry
6. Deploy the model as endpoint
7. Once done, delete the created resources